### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
%matplotlib inline

### Load Iris Data

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

### Display Data

In [3]:
X.shape

(150, 4)

In [4]:
X[:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [5]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### Split Data

Below, we are splitting the data into a train and test or holdout set. Only use the test set to evaluate your final model.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.33, random_state=42, stratify=y)

### Verifying Class Percentages

Notice that the percentage of class 0, 1, 2 are the same in the original datset as in the split set.

In [7]:
pd.Series(y).value_counts() / y.shape[0]

0    0.333333
1    0.333333
2    0.333333
dtype: float64

In [8]:
pd.Series(y_train).value_counts() / y_train.shape[0]

0    0.34
2    0.33
1    0.33
dtype: float64

### KFold

Here, we are building 3 KNN models

- a model with k=3
- a model with k=11
- a model with k=21

Each model is trained 5 times. The training happens on 4 subsets and evaluation happens on the 5th subset. We then take the average of all 5 runs and average them into a single value. This value represents the mean accuracy of the KNN at a particular "k" value.

In [9]:
kf = KFold(n_splits=5)
scores = np.zeros(3)
params = [3, 11, 21]
for i, k in enumerate(params):
    knn = KNeighborsClassifier(n_neighbors=k)
    temp = np.zeros(5)
    for j, (train_index, val_index) in enumerate(kf.split(X_train)):
        X_tr = X_train[train_index]
        y_tr = y_train[train_index]
        X_vl = X_train[val_index]
        y_vl = y_train[val_index]
    
        knn.fit(X_tr, y_tr)
        accuracy = knn.score(X_vl, y_vl)
        temp[j] = accuracy
        
    scores[i] = temp.mean()
print(scores)

[0.93 0.96 0.93]


### Choosing the best hyperparameter

Now that we have 3 average accuracy scores, we can find the hyperparameter that performed the best, i.e., had the highest accuracy.

In [10]:
best = params[scores.argmax()]; best

11

### Final Model

We now have the best hyperparameter. So we build the final model on all the training data and evaluate it on the test or holdout set to see how the model performs on data it has never seen.

In [11]:
final = KNeighborsClassifier(n_neighbors=best).fit(X_train, y_train)
final.score(X_test, y_test)

0.96

## More than one hyperparameter? Try `GridSearchCV`
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html


In [12]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)
sorted(clf.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_C',
 'param_kernel',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']